In [2]:
import pandas as pd

In [2]:
def clean_flight_data(year):

    file = "raw_data/" + str(year) + ".csv"

    original_df = pd.read_csv(file, sep=",", header=0, index_col=False,
                              usecols=[0,3,4,5,6,7,12,13,14,16,21,23],
                              na_filter = True)\
                             .reset_index(drop=True)\
                             .fillna(0)\
                             .astype({'WEATHER_DELAY': 'int32'})
    
    filtered_df = original_df.loc[((original_df['CANCELLATION_CODE'] == 'B') |
                                   (original_df['WEATHER_DELAY']      >  0)), :]

    filtered_df.loc[filtered_df['CANCELLATION_CODE'] == 'B', 'WEATHER_DELAY'] = 'CANCELLED'
    filtered_df = filtered_df.drop(columns = 'CANCELLATION_CODE')
    
    # Give columns more intuitive names
    filtered_df = filtered_df.rename(columns={'FL_DATE' : 'Date',
                                        'ORIGIN'        : 'Origin',
                                        'DEST'          : 'Destination',
                                        'CRS_DEP_TIME'  : 'Expected Departure Time',
                                        'DEP_TIME'      : 'Actual Departure Time',
                                        'DEP_DELAY'     : 'Departure Delay',
                                        'CRS_ARR_TIME'  : 'Expected Arrival Time',
                                        'ARR_TIME'      : 'Arrival Time',
                                        'ARR_DELAY'     : 'Arrival Delay',
                                        'DISTANCE'      : 'Distance',
                                        'WEATHER_DELAY' : 'Weather Delay'})
    cancelled_only_df = filtered_df.loc[filtered_df['Weather Delay'] == 'CANCELLED', :]

    cancelled_only_df = cancelled_only_df[['Date', 'Origin', 'Destination', 'Expected Departure Time',\
                                       'Expected Arrival Time', 'Distance']]
    return cancelled_only_df

In [18]:
weather_cancels_only_df = clean_flight_data(2009)        ## Takes about 7 minutes ##
for year in range(2010,2019):
    weather_cancels_only_df = pd.concat([weather_cancels_only_df, clean_flight_data(year)], axis=0, ignore_index=True)

C:\Users\itsme\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [19]:
output_path = 'clean_data/weather_cancels_only.csv'
weather_cancels_only_df.to_csv(output_path, index=False)